In [10]:
from ase_interface import ANIENS
from ase_interface import ensemblemolecule_multigpu
from ase_interface import ensemblemolecule

import pyNeuroChem as pync
import pyaniasetools as pya
import hdnntools as hdt

import numpy as np
import  ase

import time

import os
os.environ["OMP_NUM_THREADS"] = "8"

import  ase
from ase.md.langevin import Langevin
from ase.md.verlet import VelocityVerlet
from ase.io.trajectory import Trajectory
from ase import units

from ase.optimize.fire import FIRE as QuasiNewton
from ase.optimize import LBFGS

from ase.md.nvtberendsen import NVTBerendsen
from ase.md import MDLogger

from ase.io import read, write

from ase.parallel import world

In [31]:
dir = '/home/jsmith48/scratch/carbon_nano_tube/new_test/'

fname = '19_0'

# Molecule file
molfile = dir + fname + '.xyz'

# Dynamics file
xyzfile = dir + 'mdcrd.xyz'

# Trajectory file
trajfile = dir + 'traj.dat'

# Optimized structure out:
optfile = dir + 'opt_' +  fname +'.xyz'

dt = 0.2
C = 0.0001 # Optimization convergence

ntdir = '/home/jsmith48/scratch/ccsd_extrapolation/final_train_models_3/ani-1x_dft_x8ens/'
cns = ntdir + 'rHCNO-5.2R_16-3.5A_a4-8.params'
sae = ntdir + 'sae_linfit.dat'
nnf = ntdir + 'train'
Nn = 8

In [32]:
tvec = np.loadtxt('/home/jsmith48/scratch/carbon_nano_tube/new_test/'+fname+'.tvec',dtype=str)
tvec = np.array(tvec[:,1:],dtype=np.float64)

In [33]:
tvec

array([[24.885329,  0.      ,  0.      ],
       [ 0.      , 24.885329,  0.      ],
       [ 0.      ,  0.      , 12.789   ]])

In [34]:
# Load molecule
mol = read(molfile)
# L = 40.0
# mol.set_cell(([[100.0, 0, 0],
#                [0, 35.0, 0],
#                [0, 0, 35.0]]))

mol.set_cell((tvec))


mol.set_pbc((True, True, True))
#mol.set_pbc((False, False, False))

# Set NC
aens = ensemblemolecule(cns, sae, nnf, Nn, 5)
#aens = ensemblemolecule(cns, sae, nnf, Nn, 1)

# Set ANI calculator
mol.set_calculator(ANIENS(aens))

s_time = time.time()
E = mol.get_potential_energy()
F = mol.get_forces()
print('Total time:',time.time()-s_time)

print(E)

spc = mol.get_chemical_symbols()
print(len(spc))

Total time: 0.040746450424194336
-236419.61761202803
228


In [35]:
# Optimize molecule
start_time = time.time()
dyn = LBFGS(mol)
dyn.run(fmax=C,steps=3000)
print('[ANI Total time:', time.time() - start_time, 'seconds]')

# Save optimized mol
spc = mol.get_chemical_symbols()
pos = mol.get_positions(wrap=True).reshape(1,len(spc),3)


hdt.writexyzfile(optfile, pos, spc)

       Step     Time          Energy         fmax
LBFGS:    0 16:37:00  -236419.617612        0.1233
LBFGS:    1 16:37:00  -236419.644898        0.0948
LBFGS:    2 16:37:00  -236419.680602        0.1024
LBFGS:    3 16:37:00  -236420.178946        0.1340
LBFGS:    4 16:37:00  -236420.284326        0.0574
LBFGS:    5 16:37:00  -236420.289025        0.0177
LBFGS:    6 16:37:00  -236420.289365        0.0025
LBFGS:    7 16:37:00  -236420.289377        0.0012
LBFGS:    8 16:37:00  -236420.289475        0.0002
LBFGS:    9 16:37:00  -236420.289470        0.0001
LBFGS:   10 16:37:00  -236420.289472        0.0001
LBFGS:   11 16:37:00  -236420.289481        0.0002
LBFGS:   12 16:37:00  -236420.289483        0.0001
LBFGS:   13 16:37:00  -236420.289488        0.0002
LBFGS:   14 16:37:00  -236420.289466        0.0001
LBFGS:   15 16:37:00  -236420.289488        0.0004
LBFGS:   16 16:37:00  -236420.289473        0.0002
LBFGS:   17 16:37:00  -236420.289488        0.0002
LBFGS:   18 16:37:00  -236420.28

In [ ]:
# We want to run MD with constant energy using the VelocityVerlet algorithm.
#dyn = NVTBerendsen(mol, 1 * units.fs, 100.0, taut=1.0*1000*units.fs)
# coefficient to 0.04 atomic units.
dyn = Langevin(mol, dt * units.fs, 1.0 * units.kB, 0.04)

In [ ]:
# Open MD output
mdcrd = open(xyzfile,'w')

# Open MD output
traj = open(trajfile,'w')

# Define the printer
def storeenergy(a=mol, d=dyn, b=mdcrd, t=traj):  # store a reference to atoms in the definition.
    """Function to print the potential, kinetic and total energy."""
    epot = a.get_potential_energy() / len(a)
    ekin = a.get_kinetic_energy() / len(a)

    stddev =  hdt.evtokcal*a.calc.stddev

    t.write(str(d.get_number_of_steps()) + ' ' + str(ekin / (1.5 * units.kB)) + ' ' + str(epot) + ' ' + str(ekin) + ' ' + str(epot+ekin) + ' ' + str(stddev) + '\n')
    b.write(str(len(a)) + '\n' + str(ekin / (1.5 * units.kB)) + ' Step: ' + str(d.get_number_of_steps()) + '\n')
    c = a.get_positions(wrap=True)
    for j, i in zip(a, c):
        b.write(str(j.symbol) + ' ' + str(i[0]) + ' ' + str(i[1]) + ' ' + str(i[2]) + '\n')

    print('Step: %d Energy per atom: Epot = %.3f  Ekin = %.3f (T=%.3fK)  '
          'Etot = %.6f Esig = %.6f' % (d.get_number_of_steps(), epot, ekin, ekin / (1.5 * units.kB), epot + ekin, stddev))
    
# Set the printer
dyn.attach(storeenergy, interval=50)

In [ ]:
start_time = time.time()
dyn.set_temperature(50. * units.kB)
dyn.run(25000)
print('[Heat1 Total time:', time.time() - start_time, 'seconds]')

start_time = time.time()
dyn.set_temperature(100. * units.kB)
dyn.run(25000)
print('[Heat2 Total time:', time.time() - start_time, 'seconds]')

dyn.set_temperature(150. * units.kB)
dyn.run(25000)

dyn.set_temperature(200. * units.kB)
dyn.run(25000)

dyn.set_temperature(250. * units.kB)
dyn.run(25000)

dyn.set_temperature(300. * units.kB)
dyn.run(25000)

dyn.set_temperature(325. * units.kB)
dyn.run(25000)

start_time = time.time()
dyn.set_temperature(325. * units.kB)
dyn.run(250000000)
print('[ANI Total time:', time.time() - start_time, 'seconds]')